In [47]:
import pandas as pd
import numpy as np

In [48]:
attributes = ["VoW", "SoW", "CoW", "Entropy", "Label"]
bank_note_training = pd.read_csv("https://raw.githubusercontent.com/DoubekSeth/ToyDatasets/main/bank-note/train.csv", header=None, names=attributes)
bank_note_testing = pd.read_csv("https://raw.githubusercontent.com/DoubekSeth/ToyDatasets/main/bank-note/test.csv", header=None, names=attributes)

In [49]:
def add_bias_term_to_df(df):
    rows = df.shape[0]
    columns = df.shape[1]
    ones = np.ones(rows)
    #Need to check if actually need to insert
    if("Bias Term" not in df.columns):
        df.insert(columns-1, "Bias Term", ones, False)

In [50]:
def sgn(x):
    return np.where(x >= 0, 1, -1)

In [51]:
def train_perceptron(training, r, t):
    add_bias_term_to_df(training)
    weights = np.zeros(training.shape[1]-1) #Minus one from labels
    for epoch in range(t):
        print(epoch)
        for index, row in training.iterrows():
            label = row.iloc[-1]
            data = row.iloc[:len(row)-1]
            #print("Label:", label)
            #print("Data:", data)
            prediction = np.dot(weights, data)
            if((2*label-1)*prediction <= 0):
                weights = weights + r*((2*label-1)*data)
    return weights

In [52]:
def evaluate_perceptron(testing, weights):
    correct = 0
    add_bias_term_to_df(testing)
    for index, row in testing.iterrows():
        label = row.iloc[-1]
        data = row.iloc[:len(row)-1]
        prediction = np.dot(weights, data)
        if((2*label-1)*prediction > 0):
            correct += 1
    return correct/testing.shape[0]

In [53]:
def train_voted_perceptron(training, r, t):
    add_bias_term_to_df(training)
    weights = [np.zeros(training.shape[1]-1)] #Minus one from labels
    C_m = [0]
    m = 0
    for epoch in range(t):
        print(epoch)
        for index, row in training.iterrows():
            label = row.iloc[-1]
            data = row.iloc[:len(row)-1]
            prediction = np.dot(weights[m], data)
            if((2*label-1)*prediction <= 0):
                weights.append(weights[m] + r*((2*label-1)*data))
                m += 1
                C_m.append(1)
            else:
                C_m[m] = C_m[m]+1
    return weights, C_m

In [54]:
def evaluate_voted_perceptron(testing, weights, counts):
    correct = 0
    add_bias_term_to_df(testing)
    for index, row in testing.iterrows():
        label = row.iloc[-1]
        data = row.iloc[:len(row)-1]
        prediction = np.dot(counts, sgn(np.matmul(weights, data)))
        if((2*label-1)*prediction > 0):
            correct += 1
    return correct/testing.shape[0]

In [55]:
def train_average_perceptron(training, r, t):
    add_bias_term_to_df(training)
    weights = np.zeros(training.shape[1]-1) #Minus one from labels
    av = np.zeros(training.shape[1]-1)
    for epoch in range(t):
        print(epoch)
        for index, row in training.iterrows():
            label = row.iloc[-1]
            data = row.iloc[:len(row)-1]
            #print("Label:", label)
            #print("Data:", data)
            prediction = np.dot(weights, data)
            if((2*label-1)*prediction <= 0):
                weights = weights + r*((2*label-1)*data)
            av = av + weights
    return av

In [56]:
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("Normal Perceptron")
weights = train_perceptron(bank_note_training, 0.1, 10)
print(weights)
evaluate_perceptron(bank_note_testing, weights)

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Normal Perceptron
0
1
2
3
4
5
6
7
8
9
VoW         -6.108659
SoW         -4.270582
CoW         -4.030786
Entropy     -0.314627
Bias Term    5.300000
dtype: float64


0.98

In [57]:
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("Voted Perceptron")
weights, counts = train_voted_perceptron(bank_note_training, 0.1, 10)
print(weights)
print(counts)
evaluate_voted_perceptron(bank_note_testing, weights, counts)

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Voted Perceptron
0
1
2
3
4
5
6
7
8
9
[array([0., 0., 0., 0., 0.]), VoW         -0.38481
SoW         -1.01539
CoW          0.38561
Entropy      0.42228
Bias Term   -0.10000
Name: 0, dtype: float64, VoW         -0.380009
SoW         -0.855020
CoW         -0.461950
Entropy      0.346722
Bias Term   -0.200000
dtype: float64, VoW         -0.506679
SoW         -0.573190
CoW         -0.704550
Entropy      0.158102
Bias Term   -0.100000
dtype: float64, VoW         -0.711969
SoW         -0.189340
CoW         -0.784094
Entropy      0.036722
Bias Term    0.000000
dtype: float64, VoW         -1.038889
SoW         -1.463400
CoW          0.771636
Entropy      0.022540
Bias Term    0.100000
dtype: float64, VoW         -1.430929
SoW         -1.056170
CoW          0.747958
Entropy     -0.188970
Bias Term    0.200000
dtype: float64, VoW         -1.777599
SoW         -0.648930
CoW          0.319138
Entropy     -0.343150
Bias Term    0.100000
dtype: float64,

0.986

In [58]:
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("Average Perceptron")
weights = train_average_perceptron(bank_note_training, 0.1, 10)
print(weights)
evaluate_perceptron(bank_note_testing, weights)

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Average Perceptron
0
1
2
3
4
5
6
7
8
9
VoW         -40635.835084
SoW         -25346.737641
CoW         -26286.267272
Entropy      -7797.550383
Bias Term    31725.200000
dtype: float64


0.986